In [1]:
import jax.numpy as jnp
from jax import lax
import math

In [ ]:
class GammaFunction:
    """
    Gamma (parameter) function for the MotPE sampler.
    """
    def __init__(self, gamma=0.10):
        self.gamma = gamma

    def __call__(self, x):
        '''
        return number of bad samples
        '''
        return int(lax.floor(self.gamma * x))

## TPESampler
An implmentation of the Tree Parzen Estimator Sampler

### NDSort
The goal of NDsort is to assign nondomination rank to each of objective funciton results (y value).
We rank by strongly domination $$y \in \mathbb{R}^m $$
Here I provide an example on how NDSort works:
- Assume we have $\mathbb{R} = 2$ on objective function space, then given each $y_sample$

In [ ]:
class TPESampler:
    def __init__(self,
                 hyper_param,  # Hyperparameters
                 observations,  # Observations of the objective function
                 random_state,  # Random seed
                 n_EI_candidates=24,  # Number of candidates for EI maximization
                 rule='james',  # Rule to use for the next best hyperparameter to sample
                 gamma_func=GammaFunction(),  # A function to compute the gamma parameter
                 weights_func=default_weights,  # A function to compute the weights
                 split_cache=None):  # A cache of the splits
        self.hyper_param_ = hyper_param
        self.observations_ = observations
        self.random_state_ = random_state
        self.n_EI_candidates_ = n_EI_candidates
        self.rule_ = rule
        self.gamma_func_ = gamma_func # split gamma
        self.weights_func_ = weights_func
        
        # this is for split observation
        if split_cache is None:
            self.split_cache_ = {}
        else:
            self.split_cache_ = split_cache.copy()

    def NDSort(self, y_val):
        '''
        Non-dominated rank
        '''
        tmp_y_val = y_val.copy()
        NDRanks = jnp.zeros(len(y_val))

        current_rank = 0
        cnt = len(y_val)

        while cnt > 0:
            # change into 3D array with y_val.shape[0] along axis 0
            y_val_extend = jnp.tile(tmp_y_val, reps=(y_val.shape[0], 1, 1))
            y_val_swap = jnp.swapaxes(y_val_extend, 0, 1)
            dominance = jnp.sum(jnp.all(y_val_extend < y_val_swap, axis=2), axis=1)
            # assigned entry will be set to inf
            tmp_y_val.at[dominance == 0].set(jnp.finfo(jnp.float32).max)
            NDRanks.at[dominance == 0].set(current_rank)
            current_rank += 1
            cnt -= jnp.sum(dominance == 0)
        
        return NDRanks

    
    def split_observation(self, y_collection, num_bad_samples):
        '''
        Algorithm 2 in Journal
        Split the observations into good and bad observations
        '''
        # TODO: how to cache results?
        # I dont know if this will work. Maybe use pickle is a better idea
        cache_key = y_collection.tobytes()

        if cache_key in self.split_cache_:
            # bad samples indices
            D_l_sample_indices = self.split_cache_[cache_key]["l_x_i"]
            # good samples indices
            D_g_sample_indices = self.split_cache_[cache_key]["g_x_i"]      
        else:
            rank = nondominated_sort(y_collection)
            
        return D_l_sample_indices, D_g_sample_indices
        

    def sample(self):
        # Load the observed hyperparameter values and corresponding function values
        hyper_param, y_collection = self.load_hyper_param_and_y_collection()


    
    def load_hyper_param_and_y_collection(self):
        # Load the hyperparameter values
        hyper_param = []
        y_collection = []

        # TODO: this data structure can be vectorized
        for obs in self.observations_:
            if self.hyper_param_.name in obs['params']:
                # TODO convert_h
                hyper_param.append(self.convert_h(obs['params'][self.hyper_param_.name]))
                y_collection.append(obs['f_value'].values())

        jnp.asarray(hyper_param)
        jnp.asarray(y_collection, axis = 0)
        return hyper_param, y_collection
    
    # TODO: better name?
    def convert_h(self, hyper_param_val):
        try:
            lower_bound, upper_bound, _ = self.get_bound_and_q()
            if self.hyper_param_.type == 'loguniform':
                hyper_param_val = lax.log(hyper_param_val)
            return (hyper_param_val - lower_bound) / (upper_bound - lower_bound)
        except NotImplementedError:
            raise NotImplementedError('Categorical parameters do not have lower and upper options.')

    def get_bound_and_q(self):
        try:
            if self.hyper_param_.type == 'loguniform':
                return lax.log(self.hyper_param_.lower), lax.log(self.hyper_param_.upper), self.hyper_param_.q
            else:
                return self.hyper_param_.lower, self.hyper_param_.upper, self.hyper_param_.q
        except NotImplementedError:
            raise NotImplementedError('Categorical parameters do not have the log scale option.')
      